<a href="https://colab.research.google.com/github/Fet-elena/proba/blob/main/HW_2_sql_Fetisova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT (count(*) - count(crim)) crim, (count(*) - count(indus)) indus, (count(*) - count(chas)) chas, (count(*) - count(nox)) nox, (count(*) - count(zn)) zn FROM boston
    ''',
    con,
)

,crim,indus,chas,nox,zn
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT COUNT(DISTINCT crim) crim, COUNT(DISTINCT zn) zn, COUNT(DISTINCT indus) indus, COUNT(DISTINCT chas) chas, COUNT(DISTINCT nox) nox FROM boston
    ''',
    con,
)

,crim,zn,indus,chas,nox
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
tasks+=1
pd.read_sql(
    '''
    WITH MED_CRIM AS (
    SELECT CRIM FROM BOSTON ORDER BY CRIM LIMIT 1 OFFSET (SELECT (COUNT(CRIM))/2 FROM BOSTON)
    ),
    MED_ZN AS (
    SELECT ZN FROM BOSTON ORDER BY ZN LIMIT 1 OFFSET (SELECT (COUNT(ZN))/2 FROM BOSTON)
    ),
    MED_INDUS AS (
    SELECT INDUS FROM BOSTON ORDER BY INDUS LIMIT 1 OFFSET (SELECT (COUNT(INDUS))/2 FROM BOSTON)
    ),
    MED_CHAS AS (
    SELECT CHAS FROM BOSTON ORDER BY CHAS LIMIT 1 OFFSET (SELECT (COUNT(CHAS))/2 FROM BOSTON)
    ),
    MED_NOX AS (
    SELECT NOX FROM BOSTON ORDER BY NOX LIMIT 1 OFFSET (SELECT (COUNT(NOX))/2 FROM BOSTON)
    )
    SELECT *, MIN(CRIM, ZN, INDUS, CHAS, NOX) MIN FROM MED_CRIM, MED_ZN, MED_INDUS, MED_CHAS, MED_NOX
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,MIN
0,0.25915,0.0,9.69,0.0,0.538,0.0


Минимальное значение у медиан CHAS и ZN. Можно сделать вывод, что в Бостоне доля жилья в районе реки меньше, чем в другом районе. И показатель уровень жилой застройки в таблице не полностью заполнен.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами.
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
tasks+=1
pd.read_sql(
    '''
with avg_rm as (
  SELECT avg(rm) rm_max from boston where medv in (SELECT max(medv) from boston)
),
     avg_rm_min_25 as (SELECT avg(rm) rm_min_25 from boston where medv in (SELECT medv from boston ORDER BY MEDV ASC LIMIT 25)
     ),
     avg_rm_min_50 as (SELECT avg(rm) rm_min_50 from boston where medv in (SELECT medv from boston ORDER BY MEDV ASC LIMIT 50)
     ),
     avg_rm_min_100 as (SELECT avg(rm) rm_min_100 from boston where medv in (SELECT medv from boston ORDER BY MEDV ASC LIMIT 100)
     ),
     avg_rm_min_200 as (SELECT avg(rm) rm_min_200 from boston where medv in (SELECT medv from boston ORDER BY MEDV ASC LIMIT 200)
     ),
     avg_rm_min_300 as (SELECT avg(rm) rm_min_300 from boston where medv in (SELECT medv from boston ORDER BY MEDV ASC LIMIT 300)
     )
SELECT '25' cound, rm_min_25, rm_max, rm_max - rm_min_25 razn from avg_rm_min_25, avg_rm
union all
SELECT '50' cound, rm_min_50, rm_max, rm_max - rm_min_50 razn from avg_rm_min_50, avg_rm
union all
SELECT '100' cound, rm_min_100, rm_max, rm_max - rm_min_100 razn from avg_rm_min_100, avg_rm
union all
SELECT '200' cound, rm_min_200, rm_max, rm_max - rm_min_200 razn from avg_rm_min_200, avg_rm
union all
SELECT '300' cound, rm_min_300, rm_max, rm_max - rm_min_300 razn from avg_rm_min_300, avg_rm
    ''',
    con,
)

,cound,rm_min_25,rm_max,razn
0,25,5.719000,7.484,1.765000
1,50,5.757608,7.484,1.726392
2,100,5.882832,7.484,1.601168
3,200,5.913284,7.484,1.570716
4,300,5.973884,7.484,1.510116


Чем большее количество дешевых домов попадает в выборку, тем меньше разница между средним количеством комнат в дорогом и дешевом жилье.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [11]:
tasks+=1
pd.read_sql(
    '''
    with rang_lstat as (
    SELECT LSTAT, MEDV, rank() OVER (ORDER BY lstat) rank FROM boston)
    select * from rang_lstat
    where medv in (SELECT max(medv) from boston)
--    limit 5
    ''',
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


Стоимость жилья никак не говорит о статусе его владельца.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT CHAS as GRAN_REKA, AVG(MEDV) FROM boston group by CHAS
    ''',
    con,
)

,GRAN_REKA,AVG(MEDV)
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT AVG(CRIM), AVG(ZN), AVG(INDUS), AVG(CHAS), AVG(NOX) FROM boston group by CHAS
    ''',
    con,
)

,AVG(CRIM),AVG(ZN),AVG(INDUS),AVG(CHAS),AVG(NOX)
0,3.723215,11.634820,11.019193,0.0,0.551817
1,1.851670,7.714286,12.719143,1.0,0.593426


Доля промышленной застройки в районе реки немного выше, чем в отдаленности от нее. Что объясняет и повышенную долю среднего значения концентрации оксидов азота у реки.
Уровень преступности в районе реки значительно снижен. Наверное, это объясняется более элитным жильем в данном районе (наличием камер, охранников и т.д.)
Доля земли под жилую застройку возле реки ниже, что вполне понятно - прибрежную зону застраивать нельзя.


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [23]:
tasks+=1
pd.read_sql(
    """
    with prom as (
        SELECT INDUS, PERCENT_RANK() OVER(ORDER BY INDUS ASC) prom FROM boston),
    conc as (
        SELECT NOX, PERCENT_RANK() OVER(ORDER BY NOX ASC) conc FROM boston)
    SELECT 10 percintil, MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.1 and conc.conc <= 0.1
    UNION
    SELECT 20 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.2 and conc.conc <= 0.2
    UNION
    SELECT 30 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.3 and conc.conc <= 0.3
    UNION
    SELECT 40 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.4 and conc.conc <= 0.4
    UNION
    SELECT 50 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.5 and conc.conc <= 0.5
    UNION
    SELECT 60 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.6 and conc.conc <= 0.6
    UNION
    SELECT 70 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.7 and conc.conc <= 0.7
    UNION
    SELECT 80 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.8 and conc.conc <= 0.8
    UNION
    SELECT 90 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 0.9 and conc.conc <= 0.9
    UNION
    SELECT 100 , MAX(prom.INDUS) indus, MAX(conc.NOX) nox from prom, conc WHERE prom.prom <= 1 and conc.conc <= 1
    """,
    con,
)

,percintil,indus,nox
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,12.83,0.575
6,70,18.10,0.605
7,80,18.10,0.668
8,90,19.58,0.713
9,100,27.74,0.871


Концентрация оксидов азота увеличивается с ростом доли промышленной застройки

In [16]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
